# EDA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

***Load datasets and explore***

In [ ]:
loans_test = pd.read_csv("./data/raw_data/loans_test.csv")
loans_train = pd.read_csv("./data/raw_data/loans_train.csv")
loans_valid = pd.read_csv("./data/raw_data/loans_valid.csv")

# check data columns
# Correct dataset mapping
cols_train = set(loans_train.columns)  # Training set
cols_valid = set(loans_valid.columns)  # Validation set
cols_test  = set(loans_test.columns)   # Test set

common_cols = cols_train & cols_valid & cols_test
only_train  = cols_train - common_cols
only_valid  = cols_valid - common_cols
only_test   = cols_test  - common_cols
uncommon    = (cols_train | cols_valid | cols_test) - common_cols

print("Number of common columns:", len(common_cols))
print("Common columns example:", sorted(list(common_cols))[:10], "...")
print("\nColumns only in train:", sorted(list(only_train)))
print("Columns only in valid:", sorted(list(only_valid)))
print("Columns only in test:", sorted(list(only_test)))
print("\nAll uncommon columns:", sorted(list(uncommon)))


- train set(target=0)
- valid,test set(target=0,1)

In [ ]:
loans_train.describe()

In [ ]:
# === Display Data Types for All Datasets ===
import pandas as pd
import numpy as np

# Display data types for each dataset
# datasets = {'Train': loans_train, 'Valid': loans_valid, 'Test': loans_test}
datasets = {'Train': loans_train}
for name, df in datasets.items():
    print(f"\n=== {name} Dataset ===")
    print(f"Shape: {df.shape}")
    print(f"Total columns: {len(df.columns)}")
    
    # Show data types summary
    print(f"\nData types summary:")
    print(df.dtypes.value_counts())
    
    # Show detailed column information
    print(f"\nAll columns with data types:")
    print("-" * 80)
    for i, (col, dtype) in enumerate(df.dtypes.items(), 1):
        non_null_count = df[col].count()
        null_count = len(df) - non_null_count
        print(f"{i:3d}. {col:<50} | {str(dtype):<15} | Non-null: {non_null_count:5d} | Null: {null_count:5d}")
    
    # Check for problematic data types
    object_cols = df.select_dtypes(include=['object', 'category']).columns
    datetime_cols = df.select_dtypes(include=['datetime64']).columns
    
    if len(object_cols) > 0:
        print(f"\n⚠️  Object/Categorical columns ({len(object_cols)}):")
        for col in object_cols:
            print(f"  - {col}: {df[col].dtype}")
    
    if len(datetime_cols) > 0:
        print(f"\n⚠️  Datetime columns ({len(datetime_cols)}):")
        for col in datetime_cols:
            print(f"  - {col}: {df[col].dtype}")
    
    print("-" * 80)

print("\n✅ Data types analysis completed!")

***Data type check***

In [ ]:
# Convert date columns to datetime format

# Convert FirstPaymentDate and MaturityDate
for col in ['FirstPaymentDate', 'MaturityDate']:
    if col in loans_train.columns:
        loans_train[col] = pd.to_datetime(loans_train[col])
        loans_valid[col] = pd.to_datetime(loans_valid[col])
        loans_test[col] = pd.to_datetime(loans_test[col])

# Convert MonthlyReportingPeriod columns (YYYYMM format)
monthly_cols = [col for col in loans_train.columns if 'MonthlyReportingPeriod' in col]
for col in monthly_cols:
    # Convert YYYYMM format to datetime
    loans_train[col] = pd.to_datetime(loans_train[col], format='%Y%m')
    loans_valid[col] = pd.to_datetime(loans_valid[col], format='%Y%m')
    loans_test[col] = pd.to_datetime(loans_test[col], format='%Y%m')

print('converted columns: FirstPaymentDate, MaturityDate, MonthlyReportingPeriod')

***Drop columns with 100% missing data***

In [ ]:
# Comprehensive analysis of data vacancy

# 1. Overall missing data summary
missing_summary = loans_train.isnull().sum().sort_values(ascending=False)
missing_pct = (missing_summary / len(loans_train)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_summary,
    'Missing_Percentage': missing_pct
})

print(f"Total features: {len(loans_train.columns)}")
print(f"Features with missing data: {(missing_summary > 0).sum()}")
print(f"Features with complete data: {(missing_summary == 0).sum()}")

# Show top 20 features with most missing data
print("Top 20 features with most missing data:")
print(missing_df.head(5).to_string())


In [ ]:
# Drop columns with 100% missing data directly
columns_to_drop = ['ReliefRefinanceIndicator', 'PreHARP_Flag']

# Drop columns directly from original datasets
loans_train.drop(columns=columns_to_drop, inplace=True)
loans_valid.drop(columns=columns_to_drop, inplace=True)
loans_test.drop(columns=columns_to_drop, inplace=True)

***process missing data(999)***

 - CreditScore: Values outside range or missing coded as 9999.
 - MI_Pct: 999 = not available.
 - OriginalDTI: Values > 65% or missing coded as 999.
 - OriginalLTV: Invalid coded as 999.
 - N_EstimatedLTV: Range 1–998, with 999 = unknown.

In [ ]:
# CreditScore

def fix_credit_score_precision(df):
    # Step 1: clean invalid codes
    df['CreditScore'] = df['CreditScore'].replace({9999: np.nan})
    df.loc[(df['CreditScore'] < 300) | (df['CreditScore'] > 850), 'CreditScore'] = np.nan

    # Step 2: create a missing indicator (important for precision)
    df['CreditScore_missing'] = df['CreditScore'].isna().astype(int)

    # Step 3: grouped median imputation
    if 'LoanPurpose' in df.columns:
        df['CreditScore'] = df.groupby('LoanPurpose')['CreditScore'].transform(
            lambda x: x.fillna(x.median())
        )
    else:
        df['CreditScore'] = df['CreditScore'].fillna(df['CreditScore'].median())

    return df

# Apply
loans_train = fix_credit_score_precision(loans_train)
loans_valid = fix_credit_score_precision(loans_valid)
loans_test  = fix_credit_score_precision(loans_test)


In [ ]:
# MI_Pct

def fix_mi_pct_precision(df):
    # 1. Clean invalid codes
    df['MI_Pct'] = df['MI_Pct'].replace(999, np.nan)

    # 2. Add missing indicator (important for precision)
    df['MI_Pct_missing'] = df['MI_Pct'].isna().astype(int)

    # 3. Grouped median imputation — use correlated features if available
    group_cols = [col for col in ['PropertyState', 'LoanPurpose', 'ServicerName'] if col in df.columns]
    if group_cols:
        df['MI_Pct'] = df.groupby(group_cols)['MI_Pct'].transform(lambda x: x.fillna(x.median()))
    else:
        df['MI_Pct'] = df['MI_Pct'].fillna(df['MI_Pct'].median())

    return df

# Apply
loans_train = fix_mi_pct_precision(loans_train)
loans_valid = fix_mi_pct_precision(loans_valid)
loans_test  = fix_mi_pct_precision(loans_test)


In [ ]:
# OriginalDTI

def fix_dti_precision(df):
    # 1. Replace invalid values with NaN
    df.loc[df['OriginalDTI'] > 65, 'OriginalDTI'] = np.nan
    df['OriginalDTI'] = df['OriginalDTI'].replace(999, np.nan)

    # 2. Add missing indicator
    df['OriginalDTI_missing'] = df['OriginalDTI'].isna().astype(int)

    # 3. Grouped median imputation — keep contextual patterns
    group_cols = [col for col in ['LoanPurpose', 'PropertyState'] if col in df.columns]
    if group_cols:
        df['OriginalDTI'] = df.groupby(group_cols)['OriginalDTI'].transform(lambda x: x.fillna(x.median()))
    else:
        df['OriginalDTI'] = df['OriginalDTI'].fillna(df['OriginalDTI'].median())

    return df

# Apply
loans_train = fix_dti_precision(loans_train)
loans_valid = fix_dti_precision(loans_valid)
loans_test  = fix_dti_precision(loans_test)


In [ ]:
# OriginalLTV

def fix_ltv_precision(df):
    # 1. Replace invalid values
    df['OriginalLTV'] = df['OriginalLTV'].replace(999, np.nan)
    df.loc[df['OriginalLTV'] > 100, 'OriginalLTV'] = np.nan

    # 2. Add missing flag
    df['OriginalLTV_missing'] = df['OriginalLTV'].isna().astype(int)

    # 3. Group-based median imputation
    group_cols = [col for col in ['PropertyState', 'LoanPurpose', 'ServicerName'] if col in df.columns]
    if group_cols:
        df['OriginalLTV'] = df.groupby(group_cols)['OriginalLTV'].transform(lambda x: x.fillna(x.median()))
    else:
        df['OriginalLTV'] = df['OriginalLTV'].fillna(df['OriginalLTV'].median())

    return df

# Apply
loans_train = fix_ltv_precision(loans_train)
loans_valid = fix_ltv_precision(loans_valid)
loans_test  = fix_ltv_precision(loans_test)


In [ ]:
# N_EstimatedLTV

def fix_estimated_ltv_precision(df, time_points=14):
    """
    Precision-oriented cleaning for N_EstimatedLTV panel features.
    """
    for t in range(time_points):
        col = f"{t}_EstimatedLTV"
        if col not in df.columns:
            continue

        # 1. Replace invalid values
        df[col] = df[col].replace(999, np.nan)
        df.loc[df[col] > 150, col] = np.nan

        # 2. Create missing indicator
        df[f"{col}_missing"] = df[col].isna().astype(int)

    # 3. Row-wise imputation: fill each loan’s missing months
    time_cols = [f"{t}_EstimatedLTV" for t in range(time_points) if f"{t}_EstimatedLTV" in df.columns]
    df[time_cols] = df[time_cols].apply(
        lambda row: row.fillna(row.median()), axis=1
    )

    # 4. If still NaN (all months missing), fill with global median
    df[time_cols] = df[time_cols].fillna(df[time_cols].median())

    return df

# Apply
loans_train = fix_estimated_ltv_precision(loans_train)
loans_valid = fix_estimated_ltv_precision(loans_valid)
loans_test  = fix_estimated_ltv_precision(loans_test)


***Distribution Analysis***

In [ ]:
# Distribution Analysis - Non-Object Type Columns

columns_to_drop = ['index']  # Add columns to exclude from analysis

# Get all columns
all_cols = loans_train.columns.tolist()

# Select only non-object type columns (numerical, datetime, etc.)
non_object_cols = loans_train.select_dtypes(exclude=['object']).columns.tolist()

# Remove columns in the drop list
non_object_cols = [col for col in non_object_cols if col not in columns_to_drop]

print(f"- Total columns: {len(all_cols)}")
print(f"- Non-object type columns: {len(non_object_cols)}")
print(f"- Columns dropped from analysis: {columns_to_drop}")
print(f"- Object type columns (excluded): {len(all_cols) - len(non_object_cols)}")

# Group by data type
data_types = {}
for col in non_object_cols:
    dtype = str(loans_train[col].dtype)
    if dtype not in data_types:
        data_types[dtype] = []
    data_types[dtype].append(col)


# 3. Data range analysis for numerical columns
numerical_cols = loans_train[non_object_cols].select_dtypes(include=[np.number]).columns.tolist()
if len(numerical_cols) > 0:
    print("3. Data Range Analysis for Numerical Columns:")
    numerical_stats = loans_train[numerical_cols].describe()
    
    for col in numerical_cols:
        min_val = numerical_stats.loc['min', col]
        max_val = numerical_stats.loc['max', col]
        mean_val = numerical_stats.loc['mean', col]
        std_val = numerical_stats.loc['std', col]
        
        print(f"- {col}:")
        print(f"  Range: [{min_val:.2f}, {max_val:.2f}]")
        print(f"  Mean: {mean_val:.2f}, Std: {std_val:.2f}")
    print()

# 4. Distribution histograms for numerical columns
if len(numerical_cols) > 0:
    print("4. Distribution Histograms for Numerical Columns:")
    
    # Calculate subplot layout
    n_cols = len(numerical_cols)
    n_rows = (n_cols + 2) // 3  # 3 columns per row
    
    fig, axes = plt.subplots(n_rows, 3, figsize=(15, 5*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, col in enumerate(numerical_cols):
        row = i // 3
        col_idx = i % 3
        
        # Create histogram
        axes[row, col_idx].hist(loans_train[col].dropna(), bins=30, alpha=0.7, edgecolor='black')
        axes[row, col_idx].set_title(f'{col}\nRange: [{loans_train[col].min():.2f}, {loans_train[col].max():.2f}]')
        axes[row, col_idx].set_xlabel(col)
        axes[row, col_idx].set_ylabel('Frequency')
        axes[row, col_idx].grid(True, alpha=0.3)
    
    # Hide empty subplots
    for i in range(n_cols, n_rows * 3):
        row = i // 3
        col_idx = i % 3
        axes[row, col_idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()
else:
    print("No numerical columns found.")

# 5. Summary statistics
print("5. Summary Statistics:")
print(f"- Total non-object columns analyzed: {len(non_object_cols)}")
print(f"- Numerical columns: {len(numerical_cols)}")

# Show basic statistics for numerical columns
if len(numerical_cols) > 0:
    print("\nBasic Statistics for Numerical Columns:")
    print(loans_train[numerical_cols].describe().round(2))
else:
    print("- No numerical columns found for analysis.")

In [ ]:
# Output loans_train static columns and panel columns

print("=== loans_train Column Structure Analysis ===")
print(f"Total columns: {len(loans_train.columns)}")

# Static features (non-time series features)
static_features = [col for col in loans_train.columns 
                   if not col.startswith(('0_', '1_', '2_', '3_', '4_', '5_', '6_', '7_', '8_', '9_', '10_', '11_', '12_', '13_')) 
                   and col not in ['index', 'target']]

# Time series features (panel features)
panel_features = [col for col in loans_train.columns 
                  if col.startswith(('0_', '1_', '2_', '3_', '4_', '5_', '6_', '7_', '8_', '9_', '10_', '11_', '12_', '13_'))]

# Special columns
special_cols = ['index', 'target']

print("1. Static Features:")
print(f"   Count: {len(static_features)}")
print("   Column names:")
for i, col in enumerate(sorted(static_features), 1):
    print(f"   {i:2d}. {col}")


print("2. Time Series Features (Panel Features):")
print(f"   Count: {len(panel_features)}")
print("   Grouped by time points:")
panel_dict = {}
for col in sorted(panel_features):
    time_point = col.split('_')[0]
    if time_point not in panel_dict:
        panel_dict[time_point] = []
    panel_dict[time_point].append(col)

for time_point in sorted(panel_dict.keys(), key=lambda x: int(x)):
    print(f"   Time point {time_point}:")
    for col in sorted(panel_dict[time_point]):
        feature_name = col.split('_', 1)[1]  # Get feature name
        print(f"     - {feature_name}")

print("3. Special Columns:")
print(f"   Count: {len(special_cols)}")
for i, col in enumerate(special_cols, 1):
    print(f"   {i}. {col}")

print("4. Summary Statistics:")
print(f"   - Static features: {len(static_features)} columns")
print(f"   - Time series features: {len(panel_features)} columns")
print(f"   - Special columns: {len(special_cols)} columns")
print(f"   - Total: {len(static_features) + len(panel_features) + len(special_cols)} columns")
print(f"   - Time points: {len(panel_dict)} (0-{max([int(k) for k in panel_dict.keys()])})")
print(f"   - Features per time point: {len(panel_features) // len(panel_dict)}")


# Feature Engineering

***One-hot encode for Object***

In [ ]:
# Display all object columns sorted by unique values (ascending)
print("=== All Object Columns (Sorted by Unique Values) ===")
object_cols = loans_train.select_dtypes(include=['object']).columns.tolist()
print(f"Found {len(object_cols)} object columns:")
print()

# Create list of columns with their unique counts for sorting
col_info = []
for col in object_cols:
    unique_count = loans_train[col].nunique()
    col_info.append((col, unique_count))

# Sort by unique values (ascending)
col_info.sort(key=lambda x: x[1])

for i, (col, unique_count) in enumerate(col_info, 1):
    print(f"{i}. {col}")
    print(f"   Data type: {loans_train[col].dtype}")
    print(f"   Unique values: {unique_count}")
    print()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# One-Hot Encoding for categorical columns (sklearn, single-DataFrame API)
class SklearnOHE:
    def __init__(self, categorical_cols):
        self.categorical_cols = list(categorical_cols)
        self.encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.int8)
        self.feature_names_ = None
        self.existing_cols_ = None
        self.fitted_ = False

    def fit(self, df):
        # Keep only columns that exist in df
        self.existing_cols_ = [c for c in self.categorical_cols if c in df.columns]
        X_cat = df[self.existing_cols_].astype('object')
        self.encoder.fit(X_cat)
        self.feature_names_ = self.encoder.get_feature_names_out(self.existing_cols_)
        self.fitted_ = True
        return self

    def transform(self, df):
        assert self.fitted_, "Encoder is not fitted. Call fit(df) first."
        X_cat = df[self.existing_cols_].astype('object')
        ohe = pd.DataFrame(self.encoder.transform(X_cat), columns=self.feature_names_, index=df.index)
        df_out = df.drop(columns=self.existing_cols_)
        df_out = pd.concat([df_out, ohe], axis=1)
        return df_out

    def fit_transform(self, df):
        return self.fit(df).transform(df)

# Specify columns to encode
one_hot_columns = [
    'PPM_Flag', 'ProductType', 'SuperConformingFlag', 'InterestOnlyFlag', 'FirstTimeHomebuyerFlag', 'OccupancyStatus', 'Channel', 'LoanPurpose', 'ProgramIndicator', 'BalloonIndicator', 'PropertyType'
]

print("=== One-Hot Encoding for Categorical Columns (sklearn) ===")
ohe = SklearnOHE(one_hot_columns)
# Call three times: train (fit), valid (transform), test (transform)
loans_train = ohe.fit_transform(loans_train)
loans_valid = ohe.transform(loans_valid)
loans_test = ohe.transform(loans_test)


In [ ]:
def encode_categorical_fixed(train_df, valid_df, test_df):
    """
    Fixed Top-N one-hot + frequency encoding for categorical columns
    (ServicerName, SellerName, PropertyState).
    """
    top_cats = {
        'ServicerName': [
            'Other servicers', 'NATIONSTAR MORTGAGE LLC DBA MR. COOPER',
            'AMERIHOME MORTGAGE COMPANY, LLC', 'UNITED WHOLESALE MORTGAGE, LLC',
            'PENNYMAC LOAN SERVICES, LLC', 'LAKEVIEW LOAN SERVICING, LLC',
            'ROCKET MORTGAGE, LLC', 'NEW RESIDENTIAL MORTGAGE LLC',
            'JPMORGAN CHASE BANK, NATIONAL ASSOCIATION', 'FREEDOM MORTGAGE CORPORATION'
        ],
        'SellerName': [
            'Other sellers', 'UNITED WHOLESALE MORTGAGE, LLC',
            'AMERIHOME MORTGAGE COMPANY, LLC', 'ROCKET MORTGAGE, LLC',
            'PENNYMAC LOAN SERVICES, LLC', 'NEWREZ LLC',
            'JPMORGAN CHASE BANK, NATIONAL ASSOCIATION', 'U.S. BANK N.A.',
            'PHH MORTGAGE CORPORATION', 'FAIRWAY INDEPENDENT MORTGAGE CORPORATION'
        ],
        'PropertyState': ['TX','FL','CA','NY','IL','OH','PA','MI','NC','GA']
    }

    def encode_column(df, train_df, col, top_list):
        # frequency encoding
        freq = train_df[col].value_counts(normalize=True).to_dict()
        df[f"{col}_freq"] = df[col].map(freq).fillna(0)

        # one-hot encoding for top-N
        for cat in top_list:
            df[f"{col}_{cat[:15].replace(' ', '_')}"] = (df[col] == cat).astype(int)

        # Replace non-top values with "Other"
        df[col] = df[col].where(df[col].isin(top_list), 'Other')

    # copy to avoid modifying originals
    train, valid, test = [df.copy() for df in (train_df, valid_df, test_df)]

    # apply to each categorical column
    for col, cats in top_cats.items():
        print(f"Encoding {col}...")
        for df in [train, valid, test]:
            encode_column(df, train, col, cats)

    return train, valid, test


# Apply
print(f"Before encoding: Train {loans_train.shape}, Valid {loans_valid.shape}, Test {loans_test.shape}")

loans_train, loans_valid, loans_test = encode_categorical_fixed(loans_train, loans_valid, loans_test)

print(f"After encoding:  Train {loans_train.shape}, Valid {loans_valid.shape}, Test {loans_test.shape}")


### panel data

In [ ]:
# Extract statistical features from panel data time series
import numpy as np
from scipy import stats
import pandas as pd

def extract_time_series_features(df, base_variables, time_points=14):
    """
    Extract statistical features from panel data time series.
    """
    
    print(f"Extracting time series features for {len(base_variables)} variables...")
    print(f"Base variables: {base_variables}")
    
    # Create a copy to store results
    result_df = df.copy()
    
    for base_var in base_variables:
        print(f"\nProcessing {base_var}...")
        
        # Get all time-series columns for this variable
        time_cols = [f"{t}_{base_var}" for t in range(time_points)]
        existing_cols = [col for col in time_cols if col in df.columns]
        
        if not existing_cols:
            print(f"  Warning: No time-series columns found for {base_var}")
            continue
            
        print(f"  Found {len(existing_cols)} time points")
        
        # Check data type of the first column
        first_col = existing_cols[0]
        col_dtype = df[first_col].dtype
        
        if col_dtype == 'datetime64[ns]' or str(col_dtype).startswith('datetime'):
            print(f"  Skipping {base_var} - datetime columns cannot be used for statistical calculations")
            continue
        
        # Extract data for this variable across all time points
        time_data = df[existing_cols].values  # Shape: (n_samples, n_time_points)
        
        # 1. Average Level
        # Mean
        result_df[f"{base_var}_mean"] = np.nanmean(time_data, axis=1)
        # Standard deviation
        result_df[f"{base_var}_std"] = np.nanstd(time_data, axis=1)
        
        # 2. Extreme Values
        # Minimum
        result_df[f"{base_var}_min"] = np.nanmin(time_data, axis=1)
        # Maximum
        result_df[f"{base_var}_max"] = np.nanmax(time_data, axis=1)
        
        # 3. Trend
        # Linear regression slope
        slopes = []
        first_values = []
        last_values = []
        
        for i in range(len(time_data)):
            row_data = time_data[i]
            valid_mask = ~np.isnan(row_data)
            
            if np.sum(valid_mask) >= 2:  # Need at least 2 points for slope
                valid_data = row_data[valid_mask]
                time_indices = np.arange(len(row_data))[valid_mask]
                
                # Calculate slope using linear regression
                slope, _, _, _, _ = stats.linregress(time_indices, valid_data)
                slopes.append(slope)
                
                # First and last values
                first_values.append(valid_data[0])
                last_values.append(valid_data[-1])
            else:
                slopes.append(np.nan)
                first_values.append(np.nan)
                last_values.append(np.nan)
        
        result_df[f"{base_var}_slope"] = slopes
        result_df[f"{base_var}_first"] = first_values
        result_df[f"{base_var}_last"] = last_values
        
        # 4. First-Last Difference
        result_df[f"{base_var}_diff"] = result_df[f"{base_var}_last"] - result_df[f"{base_var}_first"]
        
        # 5. Volatility
        # Range (max - min)
        result_df[f"{base_var}_range"] = result_df[f"{base_var}_max"] - result_df[f"{base_var}_min"]
        
        print(f"  Added 8 statistical features for {base_var}")
    
    # Show summary of new features
    original_cols = set(df.columns)
    new_cols = set(result_df.columns)
    added_cols = new_cols - original_cols
    
    print(f"\n=== Feature Extraction Summary ===")
    print(f"Original shape: {df.shape}")
    print(f"New shape: {result_df.shape}")
    print(f"Features added: {len(added_cols)}")
    print(f"Expected features: {len(base_variables) * 8}")
    
    return result_df

# Define the 7 panel variables to extract features from (excluding MonthlyReportingPeriod)
panel_variables = [
    'CurrentActualUPB',
    'CurrentInterestRate', 
    'CurrentNonInterestBearingUPB',
    'EstimatedLTV',
    'InterestBearingUPB',
    'LoanAge',
    'RemainingMonthsToLegalMaturity'
]

print("=== Extracting Time Series Features ===")
print(f"Panel variables: {panel_variables}")
print(f"Note: MonthlyReportingPeriod skipped (datetime type)")
print(f"Expected features per variable: 8")
print(f"Total expected new features: {len(panel_variables) * 8}")

# Extract features for training data
print(f"\n--- Processing Training Data ---")
print(f"Before: {loans_train.shape}")
loans_train_ts = extract_time_series_features(loans_train, panel_variables)
print(f"After: {loans_train_ts.shape}")

# Extract features for validation data
print(f"\n--- Processing Validation Data ---")
print(f"Before: {loans_valid.shape}")
loans_valid_ts = extract_time_series_features(loans_valid, panel_variables)
print(f"After: {loans_valid_ts.shape}")

# Extract features for test data
print(f"\n--- Processing Test Data ---")
print(f"Before: {loans_test.shape}")
loans_test_ts = extract_time_series_features(loans_test, panel_variables)
print(f"After: {loans_test_ts.shape}")

# Update the original dataframes
loans_train = loans_train_ts
loans_valid = loans_valid_ts
loans_test = loans_test_ts

print(f"\n=== Final Dataset Summary ===")
print(f"Train: {loans_train.shape}")
print(f"Valid: {loans_valid.shape}")
print(f"Test: {loans_test.shape}")

# Show sample of new features
print(f"\n=== Sample of New Time Series Features ===")
ts_features = [col for col in loans_train.columns if any(col.endswith(suffix) for suffix in ['_mean', '_std', '_min', '_max', '_slope', '_diff', '_first', '_last', '_range'])]
print(f"Total time series features: {len(ts_features)}")
print(f"Sample features: {ts_features[:10]}")

# Show sample data
print(f"\nSample data for CurrentActualUPB features:")
upb_features = [col for col in loans_train.columns if col.startswith('CurrentActualUPB_')]
print(loans_train[upb_features].head())

### Statistic Data

***Normalization***

In [ ]:
# Identify columns that need scaling
print("=== Identifying Columns for Scaling ===")

# Get all columns
all_cols = loans_train.columns.tolist()
numeric_cols = loans_train.select_dtypes(include=[np.number]).columns.tolist()

# Exclude columns that shouldn't be scaled
exclude_cols = ['index', 'target']
binary_cols = []

# Identify binary columns (0/1 only)
for col in numeric_cols:
    if col not in exclude_cols:
        unique_vals = set(loans_train[col].dropna().unique())
        if unique_vals.issubset({0, 1}):
            binary_cols.append(col)

# Columns to scale = numeric - binary - excluded
scale_cols = [col for col in numeric_cols if col not in binary_cols + exclude_cols]

# Feature Scaling using the identified columns
from sklearn.preprocessing import StandardScaler

def apply_feature_scaling(train_df, valid_df, test_df, scale_cols):
    """
    Apply feature scaling to identified numerical features.
    """
    
    # Create copies
    train_scaled = train_df.copy()
    valid_scaled = valid_df.copy()
    test_scaled = test_df.copy()
    
    # Initialize scaler
    scaler = StandardScaler()
    
    train_scaled[scale_cols] = scaler.fit_transform(train_df[scale_cols])
    valid_scaled[scale_cols] = scaler.transform(valid_df[scale_cols])
    test_scaled[scale_cols] = scaler.transform(test_df[scale_cols])
    
    print("✅ Feature scaling completed!")
    return train_scaled, valid_scaled, test_scaled, scaler

# Apply scaling to all datasets
loans_train_scaled, loans_valid_scaled, loans_test_scaled, feature_scaler = apply_feature_scaling(
    loans_train, loans_valid, loans_test, scale_cols
)

# Update original dataframes
loans_train = loans_train_scaled
loans_valid = loans_valid_scaled
loans_test = loans_test_scaled

print(f"\n=== Final Scaled Dataset Summary ===")
print(f"Train: {loans_train.shape}")
print(f"Valid: {loans_valid.shape}")
print(f"Test: {loans_test.shape}")

# Verify scaling results
print(f"\n=== Scaling Verification ===")
print("Sample scaled features (mean ≈ 0, std ≈ 1):")
sample_features = scale_cols[:5]
for col in sample_features:
    mean_val = loans_train[col].mean()
    std_val = loans_train[col].std()
    print(f"{col}: mean={mean_val:.4f}, std={std_val:.4f}")

# Show feature importance info
print(f"\n=== Feature Scaling Summary ===")
print(f"✅ Successfully scaled {len(scale_cols)} numerical features")
print(f"✅ Preserved {len(binary_cols)} binary features (not scaled)")
print(f"✅ Excluded {len(exclude_cols)} special columns (index, target)")
print(f"✅ Total features processed: {len(loans_train.columns)}")

# Show which features were scaled
print(f"\n=== Scaled Features Categories ===")
print(f"📊 Original features (13): {[col for col in scale_cols if not any(col.startswith(prefix) for prefix in ['0_', '1_', '2_', '3_', '4_', '5_', '6_', '7_', '8_', '9_', '10_', '11_', '12_', '13_', 'Current', 'Servicer', 'Seller', 'Property'])][:13]}")
print(f"📈 Time series features (98): {len([col for col in scale_cols if any(col.startswith(prefix) for prefix in ['0_', '1_', '2_', '3_', '4_', '5_', '6_', '7_', '8_', '9_', '10_', '11_', '12_', '13_'])])} features")
print(f"📊 Frequency features (3): {[col for col in scale_cols if col.endswith('_freq')]}")
print(f"📊 Statistical features (36): {len([col for col in scale_cols if any(col.endswith(suffix) for suffix in ['_mean', '_std', '_min', '_max', '_slope', '_first', '_last', '_diff', '_range'])])} features")

print(f"\n💡 Scaler object saved as 'feature_scaler' for later use")

In [ ]:
# Save processed datasets to data/feature_engineering directory
import os

# Create directory and save datasets
output_dir = "data/feature_engineering"
os.makedirs(output_dir, exist_ok=True)

# Save datasets
loans_train.to_csv(os.path.join(output_dir, "loans_train_processed.csv"), index=False)
loans_valid.to_csv(os.path.join(output_dir, "loans_valid_processed.csv"), index=False)
loans_test.to_csv(os.path.join(output_dir, "loans_test_processed.csv"), index=False)

print(f"✅ Saved 3 datasets + scaler to {output_dir}")

# Training Model 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv("data/feature_engineering/loans_train_processed.csv")
test = pd.read_csv("data/feature_engineering/loans_test_processed.csv")
valid = pd.read_csv("data/feature_engineering/loans_valid_processed.csv")

### training model preparation

In [ ]:
# Fix Datetime Columns Before Training

import pandas as pd
import numpy as np

def clean_datetime_columns(df):
    """Remove or convert datetime columns to numeric features"""
    df_copy = df.copy()
    
    # Find datetime columns
    datetime_cols = df_copy.select_dtypes(include=['datetime64[ns]', 'object']).columns
    
    # Check for datetime-like strings in object columns
    for col in df_copy.columns:
        if df_copy[col].dtype == 'object':
            try:
                pd.to_datetime(df_copy[col].iloc[0])
                datetime_cols = datetime_cols.append(pd.Index([col]))
            except:
                pass
    
    print(f"Found datetime columns: {list(datetime_cols)}")
    
    # Extract numeric features from datetime columns
    for col in datetime_cols:
        if col in df_copy.columns:
            try:
                # Convert to datetime if it's a string
                if df_copy[col].dtype == 'object':
                    df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce')
                
                # Extract year, month, day
                df_copy[f'{col}_year'] = df_copy[col].dt.year
                df_copy[f'{col}_month'] = df_copy[col].dt.month
                df_copy[f'{col}_day'] = df_copy[col].dt.day
                
                # Drop original datetime column
                df_copy.drop(columns=[col], inplace=True)
                
            except:
                # If conversion fails, just drop the column
                df_copy.drop(columns=[col], inplace=True)
                print(f"Dropped problematic column: {col}")
    
    return df_copy

# Clean all datasets
train = clean_datetime_columns(train)
valid = clean_datetime_columns(valid)
test = clean_datetime_columns(test)

In [ ]:
# Extract YYYY, MM, DD from Datetime Columns
import pandas as pd
import numpy as np

def extract_datetime_features(df):
    df_copy = df.copy()
    
    for col in ['FirstPaymentDate', 'MaturityDate']:
        if col in df_copy.columns:
            # Safely convert to datetime if not already
            df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce')

            # Extract components using datetime accessor
            df_copy[col + '_year'] = df_copy[col].dt.year
            df_copy[col + '_month'] = df_copy[col].dt.month
            df_copy[col + '_day'] = df_copy[col].dt.day

            # Drop original datetime column
            df_copy.drop(columns=[col], inplace=True)
            print(f"Extracted year, month, day from {col}")
    
    return df_copy

# Apply extraction to all datasets
train = extract_datetime_features(train)
valid = extract_datetime_features(valid)
test  = extract_datetime_features(test)

print("\n✅ Datetime extraction completed successfully!")


In [ ]:
# 1. Drop all original panel columns (0_...13_...)
panel_cols = [c for c in train.columns if any(c.startswith(f"{i}_") for i in range(14))]
train = train.drop(columns=panel_cols, errors='ignore')
valid = valid.drop(columns=panel_cols, errors='ignore')
test  = test.drop(columns=panel_cols,  errors='ignore')

# 2. Keep aggregated statistical features
# (e.g. EstimatedLTV_mean, EstimatedLTV_slope, LoanAge_range, etc.)
X_train = train.drop(['index', 'target'], axis=1)
y_train = train['target']
X_valid = valid.drop(['index', 'target'], axis=1)
y_valid = valid['target']
X_test  = test.drop(['Id'], axis=1)


In [ ]:
# Use Cleaned Datasets for Model Training

from signal import valid_signals
from sklearn.impute import SimpleImputer

# Use your cleaned datasets that already have datetime features extracted
X_train = train.drop(columns=['index', 'target'], errors='ignore')
y_train = train['target']
X_valid = valid.drop(columns=['index', 'target'], errors='ignore')
y_valid = valid['target']
X_test = test.drop(columns=['Id'], errors='ignore')

# Ensure all datasets have the same columns
common_cols = set(X_train.columns) & set(X_valid.columns) & set(X_test.columns)
X_train = X_train[list(common_cols)]
X_valid = X_valid[list(common_cols)]
X_test = X_test[list(common_cols)]

In [ ]:
print(X_train)

In [ ]:
# Fill remaining NaN values using robust methods

print("Step 2: Fill remaining NaN values")

# For numeric columns, use median imputation
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if X_train[col].isna().any():
        median_val = X_train[col].median()
        if pd.isna(median_val):  # If median is also NaN, use 0
            median_val = 0
        X_train[col] = X_train[col].fillna(median_val)
        X_valid[col] = X_valid[col].fillna(median_val)
        X_test[col] = X_test[col].fillna(median_val)

# For categorical columns, use mode imputation
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    if X_train[col].isna().any():
        mode_val = X_train[col].mode()
        if len(mode_val) > 0:
            X_train[col] = X_train[col].fillna(mode_val[0])
            X_valid[col] = X_valid[col].fillna(mode_val[0])
            X_test[col] = X_test[col].fillna(mode_val[0])
        else:
            # If no mode exists, use 'unknown'
            X_train[col] = X_train[col].fillna('unknown')
            X_valid[col] = X_valid[col].fillna('unknown')
            X_test[col] = X_test[col].fillna('unknown')

# 3. Verify the fix results
print("Step 3: Verify fix results")
print(f"NaN count in X_train: {X_train.isna().sum().sum()}")
print(f"NaN count in X_valid: {X_valid.isna().sum().sum()}")
print(f"NaN count in X_test: {X_test.isna().sum().sum()}")

### Baseline Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import roc_auc_score, precision_score, roc_curve
import matplotlib.pyplot as plt

print("=== Simplified Unsupervised Baseline (AUC + Precision + ROC) ===")
print(f"Data: X_train={X_train.shape}, X_valid={X_valid.shape}, X_test={X_test.shape}")

# --- Define Models ---
models = {
    "IsolationForest": IsolationForest(n_estimators=300, contamination=0.1, random_state=42),
    "PCA": PCA(n_components=0.95, random_state=42),
    "LOF": LocalOutlierFactor(n_neighbors=20, contamination=0.1, novelty=True)
}

results = {}

# --- Train & Evaluate ---
for name, model in models.items():
    print(f"\n--- {name} ---")

    if name == "PCA":
        model.fit(X_train)
        valid_recon = model.inverse_transform(model.transform(X_valid))
        scores = np.mean((X_valid - valid_recon) ** 2, axis=1)
    else:
        model.fit(X_train)
        scores = -model.decision_function(X_valid)

    # AUC
    auc = roc_auc_score(y_valid, scores)

    # Precision using top 5% threshold
    threshold = np.percentile(scores, 95)
    y_pred = (scores >= threshold).astype(int)
    precision = precision_score(y_valid, y_pred, zero_division=0)

    print(f"AUC = {auc:.4f} | Precision@95% = {precision:.4f}")
    results[name] = (scores, auc, precision)

# --- Plot ROC Curves ---
plt.figure(figsize=(7, 6))
for name, (scores, auc, _) in results.items():
    fpr, tpr, _ = roc_curve(y_valid, scores)
    plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")

plt.plot([0, 1], [0, 1], "k--", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves - Unsupervised Baseline")
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:
print("=== X_train Columns and Data Types ===")
for col in X_train.columns:
    print(f"{col:50s} {X_train[col].dtype}")
print(f"\nTotal columns: {len(X_train.columns)}")